In [1]:
import re
import sys
from operator import add

import findspark
from pyspark.sql import SparkSession

In [16]:
def compute_contributions(urls, rank):
    total_urls = len(urls)
    for url in urls:
        yield (url, rank / total_urls)

def parse_neighbors(urls):
    parts = re.split(r'\s+', urls)
    print((parts[0], parts[1]))
    return parts[0], parts[1]

In [3]:
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')

In [4]:
spark = SparkSession.builder\
        .appName('Spark Page Rank Test')\
        .master('spark://172.31.82.177:7077')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/17 20:54:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [20]:
lines = spark.read.text(
    'hdfs://172.31.82.177:9000/web-BerkStan.txt'
).rdd.map(lambda r: r[0])

In [36]:
# lines.show(5)

In [35]:
# filtered_lines = lines.filter(~(lines.value.contains('#')))

In [8]:
# filtered_lines.show()

+--------------+
|         value|
+--------------+
|          1\t2|
|          1\t5|
|          1\t7|
|          1\t8|
|          1\t9|
|         1\t11|
|         1\t17|
|     1\t254913|
|     1\t438238|
|254913\t255378|
|254913\t255379|
|254913\t255383|
|254913\t255384|
|254913\t255392|
|254913\t255393|
|254913\t255394|
|254913\t255396|
|254913\t255399|
|254913\t255401|
|254913\t255402|
+--------------+
only showing top 20 rows



In [18]:
parse_neighbors('1\t2')

('1', '2')


('1', '2')

In [24]:
links = lines.map(lambda urls: parse_neighbors(urls)).distinct().groupByKey()

In [26]:
links.take(10)

[('121', <pyspark.resultiterable.ResultIterable at 0x7fa50a6ae8c0>),
 ('4801', <pyspark.resultiterable.ResultIterable at 0x7fa50a6ae9b0>),
 ('10866', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aead0>),
 ('18115', <pyspark.resultiterable.ResultIterable at 0x7fa50a6ae830>),
 ('18743', <pyspark.resultiterable.ResultIterable at 0x7fa50a6ae9e0>),
 ('24038', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aec20>),
 ('24473', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aeb00>),
 ('31513', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aed10>),
 ('36635', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aecb0>),
 ('43855', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aed70>)]

Exception in thread "serve RDD 38 with partitions 0" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [27]:
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [28]:
for iteration in range(10):
    contribs = links.join(ranks).flatMap(
        lambda url_urls_rank: compute_contributions(url_urls_rank[1][0], url_urls_rank[1][1])
    )

    # Update each page’s rank to be 0.15 + 0.85 * (sum of contributions)
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

In [31]:
# for (link, rank) in ranks.collect():
#     print("%s has rank: %s." % (link, rank))

In [32]:
column_names = ['link', 'rank']
ranks_df = ranks.toDF(column_names)

In [33]:
ranks_df.show()

+------+-------------------+
|  link|               rank|
+------+-------------------+
|184279|  743.4092370378007|
|    33|  660.9927237591636|
| 95552| 1235.2985405976256|
|  4507| 0.3395100631609302|
|  8786| 0.3168753543528795|
| 11743|0.20783595295189985|
| 20435|0.22789363003509644|
| 23568| 0.2977371909687734|
| 23530| 0.2977371909687734|
| 24417|0.20665739731417815|
| 24427|0.20665739731417815|
| 43412|0.34619237524637775|
| 95248| 0.6580165212328839|
| 81900|0.22140463805988567|
| 84862| 0.2640529979359341|
|100001| 0.7651243061550389|
| 82426|0.27528181734557666|
| 85611|0.30607848106727764|
| 80910|0.27325213260147807|
| 80907| 0.2670323008052145|
+------+-------------------+
only showing top 20 rows



In [34]:
spark.stop()